# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Formação Cientista de Dados</font>


## Projeto 03 - Prevendo o Nível de Satisfação dos Clientes do Santander

Este notebook contém as seguintes fases para a análise dos modelos de Machine Learning
 - Pré-Processamento
 - Criação do Modelo de Machine Learning
 - Validação e Otimização do Modelo
 - Previsão e Relatorios

## Definição do Problema de Negócio

O Banco Santander gostaria de identificar clientes insatisfeitos no início do relacionamento. Isso irá permitir que o Santander adote medidas proativas para melhorar a felicidade de um cliente antes que seja tarde demais.

Dataset: https://www.kaggle.com/c/santander-customer-satisfaction/data

### Informações sobre os atributos:

O dataset possui um grande número de variáveis numéricas e anonimas. A coluna 'TARGET' é a variável preditora

0. Clientes Satisfeitos
1. Clientes Insatisfeitos

## Extraindo e Carregando os Dados

In [ ]:
# Importando bibliotecas que serao utilizadas neste projeto
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Carregando arquivo csv usando Pandas
train = pd.read_csv("../data/train.csv", header = 0)

## Análise Exploratória de Dados

In [ ]:
# Visualizando o shape do dataset (76.020 linhas x 371 colunas)
# Neste caso algoritmo pode apresentar problemas de performance devido a alta dimensionalidade.
print(train.shape)

# Visualizando as 20 primeiras linhas do dataset
train.head(20)

In [ ]:
# Tipo de dados de cada atributo
train.dtypes

In [ ]:
# Sumário estatístico
train.describe()

In [ ]:
# Existe um problema de desbalanceamento de classes, ou seja, volume maior de um dos tipos de classe. 
# Podemos ver abaixo que existe uma clara desproporção (menos de 4% sao clientes insatisfeitos)
# entre as classes 0 (clientes satisfeitos) e 1 (clientes insatisfeitos).

# Visualizando a distribuição das classes (variavel TARGET)
pd.value_counts(train['TARGET']).plot.bar()
plt.title('TARGET histogram')
plt.xlabel('TARGET')
plt.ylabel('Frequency')

# Visualizando um df com quantidade e percentual da variavel TARGET
df = pd.DataFrame(train['TARGET'].value_counts())
df['%'] = 100*df['TARGET']/train.shape[0]
df


## Feature Engineering

Resolvendo o problema de Overfitting da variavel TARGET utilizando o OverSampling

In [ ]:
# Removendo a coluna ID do dataset
train = train.drop("ID", axis=1)

# Visualizando as 20 primeiras linhas do dataset
train.head(20)

In [ ]:
# Resolvendo problema de Overfitting utilizando o OverSampling

# Import dos módulos
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Separando o array em componentes de input e output
X = train.iloc[:,:-1]
Y = train.TARGET

# Definindo o tamanho das amostras
teste_size = 0.33

# Criando os conjuntos de dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = teste_size, random_state = 0)

# Aplicando a funcao SMOTE
# SMOTE eh um metodo de oversampling. Ele cria exemplos sinteticos da classe minoritaria ao inves de criar copias
sm = SMOTE(random_state=0)
X_treino_res, Y_treino_res = sm.fit_sample(X_treino, Y_treino.ravel())

In [ ]:
# Distribuição das classes (variavel TARGET) apos aplicar OverSampling
pd.value_counts(Y_treino_res).plot.bar()
plt.title('TARGET histogram')
plt.xlabel('TARGET')
plt.ylabel('Frequency')

# Visualizando um df com quantidade e percentual da variavel TARGET
df = pd.DataFrame(pd.value_counts(Y_treino_res), columns=['TARGET'])
df['%'] = 100*df['TARGET']/Y_treino_res.shape[0]
df

## Feature Selection - Método Ensemble

Bagged Decision Trees, como o algoritmo RandomForest (esses são chamados de Métodos Ensemble), podem ser usados para estimar a importância de cada atributo. Esse método retorna um score para cada atributo.
Quanto maior o score, maior a importância do atributo.

In [ ]:
# Importância do Atributo com o Extra Trees Classifier

# Import dos Módulos
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler

# Separando o array em componentes de input e output
X = X_treino_res
Y = Y_treino_res

# Criação do Modelo - Feature Selection
modelo = ExtraTreesClassifier()
modelo.fit(X, Y)

# Convertendo o resultado em um dataframe
df = pd.DataFrame(train.columns,columns=['Coluna'])
df['Importancia'] = pd.DataFrame(modelo.feature_importances_.astype(float))

# Realizando a ordenacao por Importancia (Maior para Menor)
result = df.sort_values('Importancia',ascending=False)

# Imprimindo as 20 variaveis mais importantes
cols_of_interest = result[1:20]['Coluna']
cols_of_interest = cols_of_interest.append(pd.Series(['TARGET']))
print(cols_of_interest)

# Deixando somente as colunas de interesse no df de treino
new_train = train[cols_of_interest]


In [ ]:
# Sumário estatístico
new_train.describe()

In [ ]:
# Verifica se existem valores nulos no novo dataset
new_train.isnull().values.any()

In [ ]:
# Pairplot
import seaborn as sns
sns.pairplot(new_train)

## Criação e Validação dos Modelos de Machine Learning

In [ ]:
# Import dos módulos
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Separando o array em componentes de input e output
X = new_train.iloc[:,:-1]
Y = new_train.TARGET

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
X = StandardScaler().fit_transform(X)

# Definindo os valores para o número de folds
num_folds = 10
seed = 10

# Preparando a lista de modelos
modelos = []
modelos.append(('LR', LogisticRegression()))
modelos.append(('LDA', LinearDiscriminantAnalysis())) # LDA: 0.960024 (0.002729)
modelos.append(('KNN', KNeighborsClassifier())) # KNN: 0.958958 (0.002947)
modelos.append(('CART', DecisionTreeClassifier())) # CART: 0.928611 (0.002662)
#modelos.append(('SVM', SVC())) # SVM: 0.960089 (0.002885)

# Avaliando cada modelo em um loop
resultados = []
nomes = []

for nome, modelo in modelos:
    kfold = KFold(n_splits = num_folds, random_state = seed)
    cv_results = cross_val_score(modelo, X, Y, cv = kfold, scoring = 'accuracy')
    resultados.append(cv_results)
    nomes.append(nome)
    msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot para comparar os algoritmos
fig = plt.figure()
fig.suptitle('Comparação de Algoritmos de Regressao')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nomes)
plt.show()

# Otimizando Performance com Métodos Ensemble

## Algoritmo XGBoost - Extreme Gradient Boosting

In [ ]:
# Import dos módulos
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

# Separando o array em componentes de input e output
X = new_train.iloc[:,:-1]
Y = new_train.TARGET

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
X = StandardScaler().fit_transform(X)

# Definindo o tamanho dos dados de treino e de teste
teste_size = 0.3
seed = 10

# Criando o dataset de treino e de teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, Y, test_size = teste_size, random_state = seed)

# Criando o modelo
modeloXGB = XGBClassifier()

# Treinando o modelo
modeloXGB.fit(X, Y)

# Fazendo previsões
y_pred = modeloXGB.predict(X_teste)
previsoes = [round(value) for value in y_pred]

# Avaliando as previsões
accuracy = accuracy_score(y_teste, previsoes)
print("Acurácia: %.2f%%" % (accuracy * 100.0))

# Curva ROC 
# A Curva ROC permite analisar a métrica AUC (Area Under the Curve).
aucResult = cross_val_score(modeloXGB, X, Y, cv = kfold, scoring = 'roc_auc')
print("AUC: %.3f" % (aucResult.mean() * 100))

# Confusion Matrix
# Permite verificar a acurácia em um formato de tabela
matrix = confusion_matrix(y_teste, previsoes)
print("Confusion Matrix")
print(matrix)

## Previsões

In [ ]:
# Relatório de Classificação

# Import dos módulos
from sklearn.metrics import classification_report

# Fazendo as previsões e construindo o relatório
report = classification_report(y_teste, previsoes)

# Imprimindo o relatório
print(report)

In [ ]:
# Criacao de função para criar um plot para a confusion matrix
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

import numpy as np

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
# Chamando a função para visualizar a confusion matrix
plot_confusion_matrix(matrix, 
                      normalize    = False,
                      target_names = ['high', 'low'],
                      title        = "Confusion Matrix")